In [1]:
using LinearAlgebra

"""
    eliminacion_gaussiana_pivoteo_escalado(A_aumentada)

Resuelve un sistema de ecuaciones lineales n x n utilizando el algoritmo de
eliminación gaussiana con pivoteo parcial escalado.

# Argumentos
- `A_aumentada::Matrix{Float64}`: La matriz aumentada del sistema [A|b] de
  tamaño n x (n+1).

# Retorna
- `Vector{Float64}`: El vector de solución `x` si existe una solución única.
- `nothing`: Si el sistema no tiene una solución única, e imprime un mensaje.
"""
function eliminacion_gaussiana_pivoteo_escalado(A_aumentada::Matrix{Float64})
    # Hacemos una copia para no modificar la matriz original
    A = copy(A_aumentada)
    n = size(A, 1) # Número de ecuaciones

    println("--- Iniciando Eliminación Gaussiana con Pivoteo Parcial Escalado ---")

    # --- INICIO DE CAMBIOS ---

    # Paso 1: Determinar los factores de escala 's' para cada fila
    s = zeros(Float64, n)
    for i in 1:n
        # s[i] es el máximo valor absoluto en la fila i (solo de la matriz de coeficientes)
        s[i] = maximum(abs.(A[i, 1:n]))
        if s[i] == 0
            println("Error: No existe una solución única (fila de ceros en la matriz de coeficientes).")
            return nothing # SALIDA
        end
    end
    # También se inicializa NROW aquí
    NROW = collect(1:n)
    
    println("Matriz Original:")
    display(A)
    println("\nFactores de escala (s): ", s, "\n")
    
    # --- PROCESO DE ELIMINACIÓN ---

    # Paso 2: Bucle principal para cada columna/pivote
    for i in 1:(n-1)
        # Paso 3: Encontrar la fila 'p' con el máximo pivote RELATIVO
        # Se calcula el ratio |A[fila, i]| / s[fila] para cada fila candidata
        ratios = [abs(A[NROW[j], i]) / s[NROW[j]] for j in i:n]
        
        _, p_local = findmax(ratios)
        p = i + p_local - 1 # Ajustamos el índice al vector NROW completo

    # --- FIN DE CAMBIOS --- (El resto del algoritmo es idéntico)

        # Verificar si el pivote absoluto es cero (aunque el ratio sea el máximo)
        if A[NROW[p], i] == 0
            println("Error: No existe una solución única.")
            return nothing
        end

        # Intercambio de fila simulado (si es necesario)
        if NROW[i] != NROW[p]
            println("Pivote en columna $i: |$(A[NROW[p], i])|. Intercambiando (simulado) fila $i con $p.")
            NROW[i], NROW[p] = NROW[p], NROW[i]
        end
        
        # Bucle para las filas debajo del pivote
        for j in (i+1):n
            # Calcular el multiplicador
            m = A[NROW[j], i] / A[NROW[i], i]

            # Ejecutar la eliminación en la fila j
            A[NROW[j], :] .-= m .* A[NROW[i], :]
        end
    end
    
    println("\n--- Matriz Modificada (forma triangular superior, filas desordenadas) ---")
    display(A)
    println("\nOrden final de las filas (NROW): ", NROW)
    println()

    # Verificar si existe una solución única después de la eliminación
    if A[NROW[n], n] == 0
        println("Error: No existe una solución única.")
        return nothing
    end

    # --- SUSTITUCIÓN HACIA ATRÁS ---

    x = zeros(Float64, n)
    x[n] = A[NROW[n], n+1] / A[NROW[n], n]

    for i in (n-1):-1:1
        suma = dot(A[NROW[i], (i+1):n], x[(i+1):n])
        x[i] = (A[NROW[i], n+1] - suma) / A[NROW[i], i]
    end

    println("--- Procedimiento completado con éxito ---")
    return x
end

eliminacion_gaussiana_pivoteo_escalado

In [2]:
# Sistema de ecuaciones:
# 30x₁ + 591400x₂ = 591700
# 5.291x₁ - 6.130x₂ = 46.78

# Matriz aumentada [A|b]
# El pivoteo simple elegiría 30, pero el escalado elegirá 5.291 porque 
# 591400 es muy grande en su propia fila.
matriz_ejemplo = [30.0    591400.0  591700.0;
                  5.291   -6.130    46.78]

# Llamamos a la nueva función
solucion = eliminacion_gaussiana_pivoteo_escalado(matriz_ejemplo)

# Imprimimos el resultado si existe
if solucion !== nothing
    println("\nLa solución del sistema es:")
    println("x₁ = ", round(solucion[1], digits=4))
    println("x₂ = ", round(solucion[2], digits=4))
end

--- Iniciando Eliminación Gaussiana con Pivoteo Parcial Escalado ---
Matriz Original:


2×3 Matrix{Float64}:
 30.0    591400.0   591700.0
  5.291      -6.13      46.78


Factores de escala (s): [591400.0, 6.13]

Pivote en columna 1: |5.291|. Intercambiando (simulado) fila 1 con 2.

--- Matriz Modificada (forma triangular superior, filas desordenadas) ---


2×3 Matrix{Float64}:
 0.0     5.91435e5   5.91435e5
 5.291  -6.13       46.78


Orden final de las filas (NROW): [2, 1]

--- Procedimiento completado con éxito ---

La solución del sistema es:
x₁ = 10.0
x₂ = 1.0
